In [88]:
## Utilities ##

import rampwf as rw
import numpy as np
import pandas as pd
from typing import Dict
from typing import Tuple
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.random_projection import SparseRandomProjection
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import pairwise_distances
from sklearn.utils.graph_shortest_path import graph_shortest_path
from scipy.stats import pearsonr, spearmanr
import multiprocessing as mp
import itertools
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor


def parallel(fn, params, n_jobs=mp.cpu_count()):
    pool = mp.Pool(n_jobs)
    print('started new process pool with {} processes'.format(n_jobs))
    try:
        res = pool.map(fn, params)
        pool.close()
        pool.join()
    except:
        print('process pool interrupted, shutting down')
        pool.terminate()
        pool.join()
        raise
    return res

mplog = mp.get_logger()

def pca(X_df, n_components=2):
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    pca = PCA(n_components=n_components)
    pca_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('pca',pca)])
    X_pc = pca_pipeline.fit_transform(X_df)    
    pca_comp_feat_ratios = np.square(pca.components_)
    feat_names = X_df.columns
    pc_feat_contrib = pd.DataFrame(pca_comp_feat_ratios, columns=feat_names)
    return X_pc #, pc_feat_contrib, pca.components_, pca.explained_variance_ratio_

def rand_projection(X_df, n_components='auto', eps=0.1):
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    proj = SparseRandomProjection(n_components=n_components, eps=eps)
    proj_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('proj', proj)])
    X_rp = proj_pipeline.fit_transform(X_df)
    return X_rp #, proj.n_components_

def umap(X_df, n_components=2, y=None, n_neighbors=5, min_dist=0.1, metric='correlation'):
    import warnings
    warnings.filterwarnings('ignore')
    imputer = Imputer(strategy='median')
    scaler = StandardScaler()
    umap = UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, metric=metric)
    umap_pipeline = Pipeline([('med_imputer', imputer),('scaler', scaler),('umap', umap)])
    X_umap = umap_pipeline.fit_transform(X_df, y)
    warnings.resetwarnings()
    return X_umap

In [84]:
# Cross Validation Generator

class TimeSeries(object):
    def __init__(self, n_cv=8, cv_block_size=0.5, period=12, unit='',
                 unit_2=None):
        self.n_cv = n_cv
        self.cv_block_size = cv_block_size
        self.period = period
        self.unit = unit
        self.unit_2 = unit_2

    def get_cv(self, X, y):
        n = len(y)
        block_size = int(
            n * self.cv_block_size / self.n_cv / self.period) * self.period
        n_common_block = n - block_size * self.n_cv
        n_validation = n - n_common_block
        #if self.unit_2 is None:
            #print('length of common block: {} {}s'.format(
             #   n_common_block, self.unit))
            #print('length of validation block: {} {}s'.format(
             #   n_validation, self.unit))
            #print('length of each cv block: {} {}s'.format(
             #  block_size, self.unit))
        #else:
            #print('length of common block: {} {}s = {} {}s'.format(
             #   n_common_block, self.unit, n_common_block / self.period,
             #   self.unit_2))
            #print('length of validation block: {} {}s = {} {}s'.format(
             #   n_validation, self.unit, n_validation / self.period,
             #   self. unit_2))
            #print('length of each cv block: {} {}s = {} {}s'.format(
             #   block_size, self.unit, block_size / self.period, self.unit_2))
        for i in range(self.n_cv):
            train_is = np.arange(0, n_common_block + i * block_size)
            test_is = np.arange(n_common_block + i * block_size, n)
            yield (train_is, test_is)
            
cv = TimeSeries(n_cv=8, cv_block_size=0.5, period=12, unit='month', unit_2='year')

for train_is, test_is in cv.get_cv(X_df,y_df):
    y = X_df.iloc[train_is,:]
    print(y)
   

            0             1             2             3             4      \
0       14.056189     14.056189     14.056189     14.056189     14.056189   
1        6.881876      6.881876      6.881876      6.881876      6.881876   
2       12.792507     12.792507     12.792507     12.792507     12.792507   
3       14.318786     14.318786     14.318786     14.318786     14.318786   
4        7.663164      7.663164      7.663164      7.663164      7.663164   
5       12.626310     12.626310     12.626310     12.626310     12.626310   
6       13.818226     13.818226     13.818226     13.818226     13.818226   
7        7.801793      7.801793      7.801793      7.801793      7.801793   
8       13.194919     13.194919     13.194919     13.194919     13.194919   
9       14.762091     14.762091     14.762091     14.762091     14.762091   
10       7.825962      7.825962      7.825962      7.825962      7.825962   
11      12.712141     12.712141     12.712141     12.712141     12.712141   

              0              1              2              3      \
0         14.056189      14.056189      14.056189      14.056189   
1          6.881876       6.881876       6.881876       6.881876   
2         12.792507      12.792507      12.792507      12.792507   
3         14.318786      14.318786      14.318786      14.318786   
4          7.663164       7.663164       7.663164       7.663164   
5         12.626310      12.626310      12.626310      12.626310   
6         13.818226      13.818226      13.818226      13.818226   
7          7.801793       7.801793       7.801793       7.801793   
8         13.194919      13.194919      13.194919      13.194919   
9         14.762091      14.762091      14.762091      14.762091   
10         7.825962       7.825962       7.825962       7.825962   
11        12.712141      12.712141      12.712141      12.712141   
12        13.859657      13.859657      13.859657      13.859657   
13         7.124859       7.124859       7.12485

          0          1          2          3          4          5      \
0     14.056189  14.056189  14.056189  14.056189  14.056189  14.056189   
1      6.881876   6.881876   6.881876   6.881876   6.881876   6.881876   
2     12.792507  12.792507  12.792507  12.792507  12.792507  12.792507   
3     14.318786  14.318786  14.318786  14.318786  14.318786  14.318786   
4      7.663164   7.663164   7.663164   7.663164   7.663164   7.663164   
5     12.626310  12.626310  12.626310  12.626310  12.626310  12.626310   
6     13.818226  13.818226  13.818226  13.818226  13.818226  13.818226   
7      7.801793   7.801793   7.801793   7.801793   7.801793   7.801793   
8     13.194919  13.194919  13.194919  13.194919  13.194919  13.194919   
9     14.762091  14.762091  14.762091  14.762091  14.762091  14.762091   
10     7.825962   7.825962   7.825962   7.825962   7.825962   7.825962   
11    12.712141  12.712141  12.712141  12.712141  12.712141  12.712141   
12    13.859657  13.859657  13.859657 

In [89]:
# Homemade Forest Parameter Tuning 
def forest(X_df, y_df, cv, max_depth, max_features, n_estimators, n_components):
    mse_sum = 0
    for train_index, test_index in cv.get_cv(X_df,y_df):
        forest_regressor = Pipeline([('imputer', Imputer(strategy='median')), 
            ('scale', StandardScaler()), 
            ('dim_red', PCA(n_components = n_components)),       
            ('regressor', RandomForestRegressor(max_depth = max_depth, max_features = max_features, n_estimators = n_estimators))])

        X_train, X_test = X_df.iloc[train_index,:], X_df.iloc[test_index,:]
        y_train, y_test = y_df[train_index], y_df[test_index]

        forest_regressor.fit(X_train,y_train)
        y_pred = forest_regressor.predict(X_test)
        mse_sum += mean_squared_error(y_test, y_pred)
        
    return ((max_depth, max_features, n_estimators),np.sqrt(mse_sum/cv.n_cv))

def run_forest(p):
    d, f, ne, n = p
    return forest(X_all, y_df, cv, max_depth=d, max_features=f, n_estimators=ne, n_components = n)

# Homemade Tree Parameter Tuning 
def tree(X_df, y_df, cv, max_depth, max_features, n_components):
    mse_sum = 0
    for train_index, test_index in cv.get_cv(X_df,y_df):
        tree_regressor = Pipeline([('imputer', Imputer(strategy='median')), 
            ('scale', StandardScaler()),
            ('dim_red', PCA(n_components = n_components)), 
            ('regressor', DecisionTreeRegressor(max_depth = max_depth, max_features = max_features))])
        X_train, X_test = X_df.iloc[train_index,:], X_df.iloc[test_index,:]
        y_train, y_test = y_df[train_index], y_df[test_index]
        tree_regressor.fit(X_train,y_train)
        y_pred = tree_regressor.predict(X_test)
        mse_sum += mean_squared_error(y_test, y_pred)
        
    return ((max_depth, max_features),np.sqrt(mse_sum/cv.n_cv))

def run_tree(p):
    d, f, n = p
    return tree(X_all, y_df, cv, max_depth=d, max_features=f, n_components = n)

# Homemade SVR Parameter Tuning 
def svr(X_df, y_df, cv, C, gamma, n_components):
    mse_sum = 0
    for train_index, test_index in cv.get_cv(X_df,y_df):
        rbf_regressor = Pipeline([('imputer', Imputer(strategy='median')),
            ('scale', StandardScaler()),
            ('dim_red', PCA(n_components = n_components)), 
            ('regressor', SVR(kernel = 'rbf', C = C, gamma = gamma))])
        
        X_train, X_test = X_df.iloc[train_index,:], X_df.iloc[test_index,:]
        y_train, y_test = y_df[train_index], y_df[test_index]
        rbf_regressor.fit(X_train,y_train)
        y_pred = rbf_regressor.predict(X_test)
        mse_sum += mean_squared_error(y_test, y_pred)
        
    return ((C, gamma),np.sqrt(mse_sum/cv.n_cv))

def run_svr(p):
    c, g, n = p
    return svr(X_all, y_df, cv, C = c, gamma = g, n_components = n)

# Homemade Boosted Forest Parameter Tuning 
def boost(X_df, y_df, cv, max_depth, max_features, n_estimators, n_components):
    mse_sum = 0
    for train_index, test_index in get_cv(X_df,y_df):
        boost_regressor = Pipeline([('imputer', Imputer(strategy='median')), 
            ('scale', StandardScaler()),
            ('dim_red', PCA(n_components = n_components)), 
            ('regressor', GradientBoostingRegressor(max_depth = max_depth, max_features = max_features, n_estimators = n_estimators))])

        X_train, X_test = X_df.iloc[train_index,:], X_df.iloc[test_index,:]
        y_train, y_test = y_df[train_index], y_df[test_index]

        forest_regressor.fit(X_train,y_train)
        y_pred = forest_regressor.predict(X_test)
        mse_sum += mean_squared_error(y_test, y_pred)
        
    return ((max_depth, max_features, n_estimators),np.sqrt(mse_sum/cv.n_cv))

def run_boost(p):
    d, f, ne, n = p
    return boost(X_all, y_df, cv, max_depth=d, max_features=f, n_estimators=ne, n_components = n)

# Homemade Lasso Parameter Tuning 
def lasso(X_df, y_df, cv, alpha, n_components):
    lasso_dict = {}
    for a in alpha:
        mse_sum = 0
        for train_index, test_index in cv.get_cv(X_df,y_df):
            lasso_regressor = Pipeline([('imputer', Imputer(strategy='median')),
                ('scale', StandardScaler()),
                ('dim_red', PCA(n_components = n_components)), 
                ('regressor', Lasso(alpha = a))])

            X_train, X_test = X_df.iloc[train_index,:], X_df.iloc[test_index,:]
            y_train, y_test = y_df[train_index], y_df[test_index]

            lasso_regressor.fit(X_train,y_train)
            y_pred = lasso_regressor.predict(X_test)
            mse_sum += mean_squared_error(y_test, y_pred)

        lasso_dict[a] = np.sqrt(mse_sum/cv.n_cv)
    return lasso_dict

# Homemade Ridge Parameter Tuning 
def ridge(X_df, y_df, cv, alpha, n_components):
    ridge_dict = {}
    for a in alpha:
        mse_sum = 0
        for train_index, test_index in get_cv(X_df,y_df):
            ridge_regressor = Pipeline([('imputer', Imputer(strategy='median')),
                ('scale', StandardScaler()),
                ('dim_red', PCA(n_components = n_components)), 
                ('regressor', Ridge(alpha = a))])

            X_train, X_test = X_df.iloc[train_index,:], X_df.iloc[test_index,:]
            y_train, y_test = y_df[train_index], y_df[test_index]

            ridge_regressor.fit(X_train,y_train)
            y_pred = ridge_regressor.predict(X_test)
            mse_sum += mean_squared_error(y_test, y_pred)

        ridge_dict[a] = np.sqrt(mse_sum/cv.n_cv)
    return ridge_dict


In [87]:
# Import Data
X_all = np.load('/Users/mchifala/csci-5622-project/sea_ice/sea_ice_X.npy')
y_df = pd.DataFrame(np.load('/Users/mchifala/csci-5622-project/sea_ice/sea_ice_y.npy'))
X_all = pd.DataFrame(np.reshape(X_all, (1500, (8*39*58))))


In [86]:
reg_dict = {}

max_depth = [5,10,20,25]
max_depth_tree = [5,10,15,20,25,50,100]
max_features = ['sqrt', 'log2']
n_estimators = [10,25,50,100,200]
alpha_lasso = [.00001, .0001, .001, .01, .1, 1]
alpha_ridge = [1, 2, 5, 10, 25, 50, 100, 200, 500, 1000]
C = np.logspace(-2, 2, num=5) 
gamma = np.logspace(-4, 2, num=7)

params_forest_boost = list(itertools.product(max_depth, max_features, n_estimators))
params_tree = list(itertools.product(max_depth, max_features))
params_svr = list(itertools.product(C, gamma))

results_forest = parallel(run_forest, params_forest_boost)
reg_dict['forest_regressor'] = dict(results_forest)
np.save('sea_ice_all_tuning.npy', reg_dict)
print("Saved!")

results_tree = parallel(run_tree, params_tree)
reg_dict['tree_regressor'] = dict(results_tree)
np.save('sea_ice_all_tuning.npy', reg_dict)
print("Saved!")

results_boost = parallel(run_boost, params_forest_boost)
reg_dict['boost_regressor'] = dict(results_boost)
np.save('sea_ice_all_tuning.npy',reg_dict)
print("Saved!")

results_svr = parallel(run_svr, params_svr)
reg_dict['svr_regressor'] = dict(results_svr)
np.save('sea_ice_all_tuning.npy', reg_dict)
print("Saved!")

reg_dict['ridge_regressor'] = ridge(X_all, y_df, alpha_ridge)
np.save('sea_ice_all_tuning.npy', reg_dict)
print("Saved!")

reg_dict['lasso_regressor'] = lasso(X_all, y_df, alpha_lasso)
np.save('sea_ice_all_tuning.npy', reg_dict)
print("Saved!")

started new process pool with 12 processes
process pool interrupted, shutting down


KeyError: '[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18\n  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36\n  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54\n  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72\n  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90\n  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108\n 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126\n 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144\n 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162\n 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180\n 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198\n 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216\n 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234\n 235 236 237 238 239 240 24